# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-01-25 21:29:13 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package CUDA...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


In [ ]:
run(`git clone https://github.com/emoryr/CycleGan.js.git`)

Cloning into 'CycleGan.js'...


Process(`git clone https://github.com/emoryr/CycleGan.js.git`, ProcessExited(0))

In [ ]:
cd("CycleGan.js/")

In [2]:
download("http://efrosgans.eecs.berkeley.edu/cyclegan/datasets/horse2zebra.zip", "horse2zebra.zip")
run(`unzip -q horse2zebra.zip`)
run(`rm horse2zebra.zip`)

Process(`unzip -q horse2zebra.zip`, ProcessExited(0))

In [3]:
import Pkg; Pkg.add(["Flux", "Images", "Parameters", "Statistics", "Zygote", "Functors", "CUDA"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Calculus ────────────────── v0.5.1
   Installed ImageIO ─────────────────── v0.6.0
   Installed TiledIteration ──────────── v0.3.1
   Installed OffsetArrays ────────────── v1.10.8
   Installed TiffImages ──────────────── v0.5.3
   Installed PNGFiles ────────────────── v0.3.12
   Installed ImageSegmentation ───────── v1.7.0
   Installed AxisArrays ──────────────── v0.4.4
   Installed FFTW ────────────────────── v1.4.5
   Installed ZipFile ─────────────────── v0.9.4
   Installed NNlibCUDA ───────────────── v0.1.11
   Installed ProgressMeter ───────────── v1.7.1
   Installed ImageMagick ─────────────── v1.2.2
   Installed Sixel ───────────────────── v0.1.2
   Installed NearestNeighbors ────────── v0.4.9
   Installed NNlib ───────────────────── v0.7.34
   Installed RegionTrees ─────────────── v0.3.2
   Installed IntelOpenMP_jll ─────────── v2018.0.3+2
   Installed OpenEXR ─────────────

In [96]:
using Functors
using Base.Iterators: partition
using Flux
using Flux.Optimise: update!
using Flux: logitbinarycrossentropy
using Flux.Losses: mse, mae
using Images
using Statistics
using Parameters: @with_kw
using Random
using Printf
using Zygote
using CUDA
if has_cuda()		# Check if CUDA is available
    @info "CUDA is on"
    CUDA.allowscalar(false)
end


size_img = 180

@with_kw struct HyperParams
    batch_size::Int = 5
    epochs::Int = 100
    verbose_freq::Int = 800
    size_dataset::Int = 1000
    lr_dscr_A::Float64 = 0.00005
    lr_gen_A::Float64 = 0.00005
    lr_dscr_B::Float64 = 0.00005
    lr_gen_B::Float64 = 0.00005
end

┌ Info: CUDA is on
└ @ Main In[96]:15


HyperParams

# Netwoks.jl

In [50]:
struct ConvBlock
    conv
end

@functor ConvBlock
function ConvBlock(
    kernel_size::Int,
    in_channels::Int,
    out_channels::Int,
    act::Bool = true,
    down::Bool = true;
    kwargs...
) 
return ConvBlock(
    Chain(
        if down
            Conv((kernel_size, kernel_size), in_channels => out_channels; kwargs...)
        else
            ConvTranspose((kernel_size, kernel_size), in_channels => out_channels; kwargs...)
        end,
        InstanceNorm(out_channels),
        if act 
           x -> relu.(x) 
        else 
            identity
        end)
) 
end

function (net::ConvBlock)(x)
    return net.conv(x)
end

struct ResidualBlock
    block
end

@functor ResidualBlock
function ResidualBlock(
    in_channels::Int
) return ResidualBlock(
    Chain(
        ConvBlock(3, in_channels, in_channels,true,true; pad=1),
        ConvBlock(3, in_channels, in_channels,false,true;pad=1)
    )
) 
end

function(net::ResidualBlock)(x)
    return x + net.block(x)
end

struct Block
    conv
end

@functor Block
function Block(
    in_channels::Int,
    out_channels::Int;
    stride::Int
) return Block(
    Chain(
        Conv((4,4), in_channels=>out_channels; stride=stride, pad=1),
        InstanceNorm(out_channels),
        x -> leakyrelu.(x, 0.2)
    )
)
end

function(net::Block)(x)
    return net.conv(x)
end

# Discriminator.jl

In [32]:
struct Discriminator
    initial
    model
end

@functor Discriminator
function Discriminator(
    in_channels::Int = 3,
    features::Any = [64, 128, 256, 512]
) 
    layers = []
    channel = features[1]
    for index in range(2, length(features))
            if features[index] != last(features)
                push!(layers, 
                    Block(channel, features[index], stride=2)
                )
            else
                push!(layers, 
                    Block(channel, features[index], stride=1)
                )
            end
            channel = features[index]
    end
    push!(layers,
        Conv((4,4), channel => 1; stride=1, pad=1)    
    )
return Discriminator(
    Chain(
        Conv((4,4), in_channels=>features[1]; stride=2, pad=1),
        x -> leakyrelu.(x, 0.2)
    ),
    Chain(layers...)
) |> gpu
end

function (net::Discriminator)(x)
    input = net.initial(x)
    return sigmoid.(net.model(input))
end

using Random
function test()
    img_channels = 3
    img_size = 100
    ## need to explicity type to avoid Slow fallback implementation 
    ## https://discourse.julialang.org/t/flux-con-warning/49456
    x = randn(Float32, (img_size, img_size, img_channels, 5))
    preds = Discriminator()
    println(size(preds(x)))
end

test (generic function with 1 method)

# Generator.jl


In [51]:
struct Generator
    initial
    downblocks
    resblocks
    upblocks
    final
end

@functor Generator
function Generator(
    in_channels::Int,
    num_features::Int = 64,
    num_residual::Int = 9,
)
return Generator(
    Chain(
        Conv((7,7), in_channels => num_features; stride=1, pad=3),
        InstanceNorm(num_features),
        x -> relu.(x)
    ),
    [
        ConvBlock(3, num_features, num_features*2, true, true;stride=2 ,pad=1), 
        ConvBlock(3, num_features*2, num_features*4, true, true;stride=2 ,pad=1),  
    ],
    Chain([ResidualBlock(num_features*4) for _ in range(1, length=num_residual)]...),
    [
        ConvBlock(3, num_features*4, num_features*2,true,false; stride=2 ,pad=SamePad()), 
        ConvBlock(3, num_features*2, num_features,true,false; stride=2 ,pad=SamePad()),  
    ],
    Conv((7,7),num_features=>in_channels; stride=1,pad=3)
)  |> gpu
end

function (net::Generator)(x)
    input = net.initial(x)
    for layer in net.downblocks
        input = layer(input)
    end
    input = net.resblocks(input)
    for layer in net.upblocks
        input = layer(input)
    end
    return tanh.(net.final(input))
end

# Cyclegan.jl

In [83]:
# reading the dataset
function convertI2Float(img)
    img_resize = imresize(float.(img), (size_img,size_img))
    if length(size(img_resize)) == 2
        img_resize = RGB.(img_resize)
    end
	return permutedims(channelview(img_resize), (3,2,1))
end

function load_images(path::String, size::Int)
	images= zeros(Float32,size_img,size_img,3,size)
	for (index, img) in enumerate(readdir(path, join = true))
		images[:,:,:,index] = convertI2Float(load(img))
        if index == size
            break
        end
	end
	return images |> gpu
end

function load_data(hparams)
    # Load folder dataset
    images_A = load_images("horse2zebra/trainA/", hparams.size_dataset)
    images_B = load_images("horse2zebra/trainB/", hparams.size_dataset)
    data = [ (images_A[:,:,:, r], images_B[:,:,:, r]) |> gpu for r in partition(1:hparams.size_dataset, hparams.batch_size)]
    return data
end

load_data (generic function with 1 method)

In [84]:
# Calculate errors
function calculate_loss_discr(real, fake)
    return ((real + fake) / 2)
end

function calculate_loss_gen(loss_G_A, loss_G_B, cycle_A_loss, 
    cycle_B_loss)
    return (loss_G_A
        + loss_G_B
        + cycle_A_loss * 10
        + cycle_B_loss * 10)
end

calculate_loss_gen (generic function with 1 method)

In [85]:
# train discr
function train_discr(discr_A, discr_B, original_A, original_B,
                     fake_A, fake_B, opt_discr)
    ps = Flux.params(discr_A, discr_B)
    loss, back = Zygote.pullback(ps) do
                        #calculate A
                        D_A_real = discr_A(original_A)
                        D_A_fake = discr_A(fake_A)
                        D_A_real_loss = mse(D_A_real, gpu(ones(size(D_A_real))))
                        D_A_fake_loss = mse(D_A_fake, gpu(zeros(size(D_A_fake)))) 
                        D_A_loss = D_A_real_loss + D_A_fake_loss

                        #calculate A
                        D_B_real = discr_B(original_B)
                        D_B_fake = discr_B(fake_B)
                        D_B_real_loss = mse(D_B_real, gpu(ones(size(D_B_real))))
                        D_B_fake_loss = mse(D_B_fake, gpu(zeros(size(D_B_fake))))
                        D_B_loss = D_B_real_loss + D_B_fake_loss
                        calculate_loss_discr(D_A_loss, D_B_loss)
    end
    grads = back(1.f0)
    update!(opt_discr, ps, grads)

    return loss
end

Zygote.@nograd train_discr

In [86]:
#train gen
function train_gan(gen_A, gen_B, discr_A, discr_B, original_A, original_B, opt_gen, opt_discr)
    loss = Dict()
    ps = Flux.params(gen_A, gen_B)
    loss["G_loss"], back = Zygote.pullback(ps) do
                            fake_A = gen_A(original_B)
                            fake_B = gen_B(original_A)
                            loss["D_loss"]= train_discr(discr_A, discr_B,
                                                        original_A, original_B,
                                                        fake_A, fake_B, opt_discr)
                            # adversarial loss for both generators
                            D_A_fake = discr_A(fake_A)
                            D_B_fake = discr_B(fake_B)
                            loss_G_A = mse(D_A_fake, gpu(ones(size(D_A_fake))))
                            loss_G_B = mse(D_B_fake, gpu(ones(size(D_B_fake))))

                            #cycle loss
                            cycle_B = gen_B(fake_A)
                            cycle_A = gen_A(fake_B)
                            cycle_B_loss = mae(original_B, cycle_B)
                            cycle_A_loss = mae(original_A, cycle_A)

                            # identity loss (remove these for efficiency if you set lambda_identity=0)
                            #identity_B = gen_B(original_B)
                            #identity_A = gen_A(original_A)
                            #identity_B_loss = mae(original_B, identity_B)
                            #identity_A_loss = mae(original_A, identity_A)
                            
                            calculate_loss_gen(loss_G_A, loss_G_B, cycle_A_loss, cycle_B_loss)
    end
    grads = back(1.f0)
    update!(opt_gen, ps, grads)
    return loss
end

train_gan (generic function with 1 method)

In [87]:
# output function
function create_output_image(gen, image)
    @eval Flux.istraining() = false
    fake_image = cpu(gen(gpu(image)))
    @eval Flux.istraining() = true
    image_array = permutedims(dropdims(fake_image; dims=4), (3,2,1))
    image_array = colorview(RGB, image_array)
    return clamp01nan.(image_array)
end


create_output_image (generic function with 1 method)

In [88]:
#train cyclegan function
function train()
    println("STARTING TRAINING")
    hparams = HyperParams()

    data = load_data(hparams)

    #test images
    test_images_A=zeros(Float32,size_img,size_img,3,1)
    test_images_B=zeros(Float32,size_img,size_img,3,1)
    test_images_A[:,:,:,1] = convertI2Float(load("horse2zebra/testA/n02381460_1000.jpg"))
    test_images_B[:,:,:,1] = convertI2Float(load("horse2zebra/testB/n02391049_100.jpg"))

    # Discriminator
    dscr_A = Discriminator()
    dscr_B = Discriminator() 

    # Generator
    gen_A =  Generator(3, 64) |> gpu
    gen_B =  Generator(3, 64) |> gpu

    # Optimizers
    opt_dscr = ADAM(hparams.lr_dscr_A, (0.5,0.99))
    opt_gen = ADAM(hparams.lr_gen_A, (0.5,0.99))

    isdir("output")||mkdir("output")
    
    # Training
    train_steps = 0
    for ep in 1:hparams.epochs
        @info "Epoch $ep"
        for (x,y) in data
                # Update discriminator and generator
            loss = train_gan(gen_A, gen_B, dscr_A, dscr_B, x, y, opt_gen, opt_dscr)
            if train_steps % hparams.verbose_freq == 0
                @info("Train step $(ep), Discriminator loss = $(loss["D_loss"]), Generator loss = $(loss["G_loss"])")
                # Save generated fake image
                output_image_A = create_output_image(gen_A, test_images_B)
                output_image_B = create_output_image(gen_B, test_images_A)
                save(@sprintf("output/cgan_A_steps_%06d.png", train_steps), output_image_A)
                save(@sprintf("output/cgan_B_steps_%06d.png", train_steps), output_image_B)
            end
        train_steps += 1
        end
    end
    println("Finish Training")
    output_image_A = create_output_image(gen_A, test_images_B)
    output_image_B = create_output_image(gen_B, test_images_A)
    save("output/cgan_A_steps_final.png", output_image_A)
    save("output/cgan_B_steps_final.png", output_image_B)
end

train (generic function with 1 method)

In [ ]:
# run train
train()